In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#Cargamos los datos a trabajar

x_train=np.load("./data/train_signals.npy")
marks_train=np.load("./data/train_marks.npy")

x_val=np.load("./data/val_signals.npy")
marks_val=np.load("./data/val_marks.npy")

x_test=np.load("./data/test_signals.npy")
marks_test=np.load("./data/test_marks.npy")

# Funcion que genera el array de salida con ceros y unos
def getLabels(inp, marks):
    total = np.zeros((len(inp),len(inp[0])))
    for i in range(len(inp)):
        inf = marks[i][0]
        sup = marks[i][1]
        out = np.zeros(len(inp[0]))
        for j in range(sup - inf):
            out[inf + j] = 1
        total[i] = out
    return total

# Arrays de salida correspondientes
y_train = getLabels(x_train, marks_train)
y_val = getLabels(x_val, marks_val)
y_test = getLabels(x_test, marks_test)

# Expandir dimensiones?
x_train = np.expand_dims(x_train,2)
y_train = np.expand_dims(y_train,2)

x_val = np.expand_dims(x_val,2)
y_val = np.expand_dims(y_val,2)

x_test = np.expand_dims(x_test,2)
y_test = np.expand_dims(y_test,2)


# Formato tensorflow dataset
# train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
# val = tf.data.Dataset.from_tensor_slices((x_val,y_val))
# test = tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [9]:
#Modelo Propuesto
prob = 0.5

model2 = tf.keras.Sequential()

model2.add(tf.keras.layers.Conv1D(16, 7, activation='elu', input_shape = (800,1)))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Conv1D(32, 7, activation='elu'))
model2.add(tf.keras.layers.Dropout(prob))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Conv1D(48, 7, activation='elu'))
model2.add(tf.keras.layers.Dropout(prob))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Conv1D(64, 7, activation='elu'))
model2.add(tf.keras.layers.Dropout(prob))
model2.add(tf.keras.layers.MaxPool1D(pool_size=5,strides=2))

model2.add(tf.keras.layers.Flatten())

model2.add(tf.keras.layers.Dense(50, activation=tf.nn.relu, name='fc3'))
model2.add(tf.keras.layers.Dropout(0.5))

model2.add(tf.keras.layers.Dense(800, activation=tf.nn.relu, name='fc4'))

model2.add(tf.keras.layers.Softmax(axis = 1))


model2.build(input_shape=(None,200,1))

model2.compile(loss='categorical_crossentropy',
               optimizer="adam",
               metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 794, 16)           128       
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 395, 16)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 389, 32)           3616      
_________________________________________________________________
dropout_12 (Dropout)         (None, 389, 32)           0         
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 193, 32)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 187, 48)           10800     
_________________________________________________________________
dropout_13 (Dropout)         (None, 187, 48)          

In [10]:
# Falla :c

model2.fit(
    x = x_train,
    y = y_train,
    epochs=100,
#     steps_per_epoch=100,
#     validation_data= (x_val,y_val),
#     validation_freq=1,
#     callbacks=[tf.keras.callbacks.TensorBoard('./logs/dos', update_freq=100)]
)



Epoch 1/100
140/140 [==============================] - 8s 58ms/step - loss: 212422295552.0000 - mean_io_u_3: 0.3952
Epoch 2/100
140/140 [==============================] - 9s 67ms/step - loss: 52390438895616.0000 - mean_io_u_3: 0.3954
Epoch 3/100
104/140 [=====================>........] - ETA: 2s - loss: 729279407063040.0000 - mean_io_u_3: 0.3955

KeyboardInterrupt: 

In [6]:
#Creamos una funcion que muestree a un ratio deseado

def sampling(signals,samples,lenght,rate):
    x_new=[]
    for i in signals:
        x=[]
        for j in range(0,lenght,rate):
            x.append(i[j])
        x_new.append(x)
    return np.array(x_new)

#Obtenemos los nuevos resultados de nuestras señales a trabajar

x_train=sampling(x_train_pre,trainLen,800,4)
x_val=sampling(x_val_pre,valLen,800,4)
x_test=sampling(x_test_pre,testLen,800,4)

#Creamos una funcion que clasifique con 1 o 0 los valores de la señal a trabajar

def oneHot(marks,lenght,rate):
    y_new=[]
    for i in marks:
        y=[]
        for j in range(0,lenght,rate):
            if(i[0]<=j & j<=i[1]):
                y.append(1)
            else:
                y.append(0)
        y_new.append(y)
    return np.array(y_new)

#Obtenemos los nuevos resultados de nuestras marcas a trabajar

y_train=oneHot(y_train_pre,800,4)
y_val=oneHot(y_val_pre,800,4)
y_test=oneHot(y_test_pre,800,4)

x_train2=np.expand_dims(x_train,2)
y_train2=np.expand_dims(y_train,2)

x_val2=np.expand_dims(x_val,2)
y_val2=np.expand_dims(y_val,2)

x_test2=np.expand_dims(x_test,2)
y_test2=np.expand_dims(y_test,2)

BATCH_SIZE=1

training_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train2, y_train2)).shuffle(buffer_size=1024).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices(
    (x_test2, y_test2)).shuffle(buffer_size=1024).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    (x_val2, y_val2)).shuffle(buffer_size=1024).batch(BATCH_SIZE)

time=np.arange(0,800,4)

NameError: name 'x_train_pre' is not defined